In [13]:
#libraries
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend
from keras.utils import np_utils
from keras import regularizers
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.engine.input_layer import Input
from keras.models import Model
from keras.optimizers import SGD, Adam

#tensorflow2 libs
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate, ZeroPadding2D, GlobalAveragePooling2D
from keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [14]:

# dimensions of our images.
img_width, img_height = 64, 64

In [15]:
#declarations of dataset specs
train_data_dir = 'data/train'
test_data_dir = 'data/test'
nb_train_samples = 4672
nb_validation_samples = 660
epochs = 10
batch_size = 32
num_classes = 7
top_model_weights_path = 'best_model.hdf5'

In [16]:
 #adding zero padding in images
base_model = Sequential()                                                                                                                                                                              
base_model.add(ZeroPadding2D((80, 80), input_shape=(64, 64, 3)))

In [17]:

#Resnet50
new_input = Input(shape=(224, 224, 3))

# base_model = Sequential()
base_model.add(tf.keras.applications.ResNet50(input_tensor=new_input, weights='imagenet', include_top=False))
print('Model loaded.')


Model loaded.


In [18]:
#freezing layers for regularization
for i in range(len(base_model.layers)):
    base_model.layers[i].trainable = False


# add new classifier layers
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(7, activation='sigmoid'))

#add top model to base
base_model.add(top_model)

#load weights of top model
# base_model.load_weights(top_model_weights_path)
print("Created model and loaded weights from file")

base_model.summary()

Created model and loaded weights from file
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
sequential_3 (Sequential)    (None, 7)                 25692167  
Total params: 49,279,879
Trainable params: 25,692,167
Non-trainable params: 23,587,712
_________________________________________________________________


In [19]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
base_model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [20]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [21]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1. / 255)

In [22]:
#inputting the dataset
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 4672 images belonging to 7 classes.
Found 660 images belonging to 7 classes.


In [23]:
filepath="best_boi.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [24]:

#running the model
history = base_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks_list)

Epoch 1/10
21/21 [==============================] - 4s 178ms/step - loss: 0.9081 - acc: 0.3245

Epoch 00001: val_acc improved from -inf to 0.32446, saving model to best_boi.hdf5
146/146 [==============================] - 30s 206ms/step - loss: 0.5290 - acc: 0.8043 - val_loss: 0.9081 - val_acc: 0.3245
Epoch 2/10
21/21 [==============================] - 3s 131ms/step - loss: 0.8029 - acc: 0.5853

Epoch 00002: val_acc improved from 0.32446 to 0.58528, saving model to best_boi.hdf5
146/146 [==============================] - 25s 174ms/step - loss: 0.4340 - acc: 0.8349 - val_loss: 0.8029 - val_acc: 0.5853
Epoch 3/10
21/21 [==============================] - 3s 131ms/step - loss: 0.7046 - acc: 0.6232

Epoch 00003: val_acc improved from 0.58528 to 0.62316, saving model to best_boi.hdf5
146/146 [==============================] - 25s 174ms/step - loss: 0.4175 - acc: 0.8420 - val_loss: 0.7046 - val_acc: 0.6232
Epoch 4/10
21/21 [==============================] - 3s 132ms/step - loss: 0.5443 - acc: 

In [25]:
#putting data into variables
x_train, y_train = train_generator.next()
x_test, y_test = validation_generator.next()

In [26]:

#getting evaluation scores
score = base_model.evaluate(x_test, y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

32/32 [==============================] - 0s 4ms/sample - loss: 0.3783 - acc: 0.8705
Test loss: 0.3783434331417084
Test accuracy: 0.87053573


In [27]:
import matplotlib.pyplot as plt

history_dict = history.history

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

In [28]:
# #saving weights
# base_model.save_weights('fc_model.h5')